<a href="https://colab.research.google.com/github/yellowjs0304/3-min-pytorch/blob/master/05-%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%B2%98%EB%A6%AC%EB%8A%A5%EB%A0%A5%EC%9D%B4_%ED%83%81%EC%9B%94%ED%95%9C_CNN/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN으로 패션 아이템 구분하기
Convolutional Neural Network (CNN) 을 이용하여 패션아이템 구분 성능을 높여보겠습니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

from torchsummary import summary # 모델 구조를 한번에 확인하기 위해 추가한 라이브러리
#torchsummary : 파라미터 개수, 레이어 구조를 정리하여 확인할 수 있다.

In [2]:
# GPU 사용여부 확인 코드
# DEVICE 안에 cuda(gpu)가 할당되어야 함
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

## 데이터셋 불러오기

In [4]:
# Dataloader : 데이터셋을 batch 단위로 쪼개서 학습할 때 모델의 입력으로 주는 클래스.
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,#학습
                   download=True,#없으면 다운로드
                   transform=transforms.Compose([#torchvision.transform = 입력 변환 라이브러리
                       transforms.ToTensor(),#이미지를 Tensor로
                       transforms.Normalize((0.1307,), (0.3081,))#이미지 정규화
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, #테스트
                   download=True,#없으면 다운로드
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 뉴럴넷으로 Fashion MNIST 학습하기

In [5]:
class Net(nn.Module):

    # CNN Network Class 정의#
    # kernel size 5x5, 2 Convolutional layer#
    def __init__(self):
        super(Net, self).__init__()

        # 채널 수 : 1(흑백 이미지) / output volume_size : 10(필터의 개수==10개의 특징맵을 생성) / kernel_size(필터의 사이즈) = 5x5
        # kernel size를 하나만 입력하면 NxN으로 간주. / NxM을 원할경우, (N,M)으로 정의
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        
        # 입력 채널 수 : 10 (conv1의 결과물) / output volume size : 20 (20개의 특징맵을 생성) / kernel_size(필터의 사이즈) = 5x5
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        #Dropout 적용
        self.conv2_drop = nn.Dropout2d()

        # 일반 신경망을 거치면서 이전 출력 크기인 320을 기준으로  50, 10순으로 작아지도록 설정
        # 이때 50과 같은 중간 값은 임의로 설정한 값. 10은 분류해야 할 클래스의 개수(FASHION MNIST CLASS 개수)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    #CNN Network의 동작(forward)함수, 본격적으로 데이터가 지나갈 길을 닦아준다 생각하면 됨.#
    def forward(self, x):
        # 각 레이어는 conv - pooling - relu를 하나의 묶음으로 간주
        x = F.relu(F.max_pool2d(self.conv1(x), 2))# max pooling with (2x2)kernel
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))

        # 데이터를 FC레이어에 넣기 위해 2차원에서 1차원 형태로 변경
        x = x.view(-1, 320)

        #RELU activation function+Dropout
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        
        # 출력 클래스 10개인 Output생성
        x = self.fc2(x)
        return x

## 하이퍼파라미터 

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다. 일반적으로 CPU 1개만 사용할 경우 필요는 없지만, GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다. 지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [6]:
model     = Net().to(DEVICE)
summary(model, (1, 28, 28)) #모델 구조를 보기 위하여 삽입한 코드
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
print(model)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
         Dropout2d-3             [-1, 20, 8, 8]               0
            Linear-4                   [-1, 50]          16,050
            Linear-5                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.08
Estimated Total Size (MB): 0.15
----------------------------------------------------------------
Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
 

## 학습하기

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()#for문을 지날때마다 기울기 새로 계산
        output = model(data)#모델 예측값
        loss = F.cross_entropy(output, target)#예측값, 정답 간의 Cross entropy loss
        loss.backward()#역전파 기울기 계산
        optimizer.step()#weight고치기

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

## 테스트하기

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 학습이 되는지 확인해봅시다!

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299135
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.411538
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.648250
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.514119
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.344039
[1] Test Loss: 0.1586, Accuracy: 95.31%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.469092
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.499743
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.363924
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.136671
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.224638
[2] Test Loss: 0.1028, Accuracy: 96.81%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.172936
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.268849
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.381308
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.205941
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.358028
[3] Test Loss: 0.0813, Accuracy: 97.46%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.144789
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.305467
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.106295
